In [1]:
import json
import pandas as pd
import pickle
import sys
sys.path.append("../cider")
from pyciderevalcap.ciderD.ciderD import CiderD
import os
os.path.join('data', 'vizwiz-train-idxs'+'.p')

'data/vizwiz-train-idxs.p'

In [4]:
QUALITY_ISSUE_STR = 'quality issues'

In [6]:
prediction = '0502_test.json'
pred = pd.DataFrame(json.load(open(prediction, 'r')))

In [7]:
pred[pred.caption.str.contains(QUALITY_ISSUE_STR)]

,image_id,caption
53,32821,quality issues are too severe to recognize vis...
71,32839,quality issues are too severe to recognize vis...
98,32866,quality issues are too severe to recognize vis...
256,33024,quality issues are too severe to recognize vis...
367,33135,quality issues are too severe to recognize vis...
...,...,...
7645,32413,quality issues are too severe to recognize vis...
7764,32532,quality issues are too severe to recognize vis...
7841,32609,quality issues are too severe to recognize vis...
7930,32698,quality issues are too severe to recognize vis...


In [5]:
pred[pred.caption.str.contains(QUALITY_ISSUE_STR)]

,image_id,caption
71,32839,quality issues are too severe to recognize vis...
250,33018,quality issues are too severe to recognize vis...
373,33141,quality issues are too severe to recognize vis...
428,33196,quality issues are too severe to recognize vis...
622,33390,quality issues are too severe to recognize vis...
...,...,...
7645,32413,quality issues are too severe to recognize vis...
7665,32433,quality issues are too severe to recognize vis...
7681,32449,quality issues are too severe to recognize vis...
7829,32597,quality issues are too severe to recognize vis...


In [2]:
scorer = CiderD(df='vizwiz-train-idxs')

In [3]:
tj = json.load(open('../data/train.json', 'r'))
vj = json.load(open('../data/val.json', 'r'))

In [4]:
yt_g = pd.DataFrame(tj['annotations'])
yv_g = pd.DataFrame(vj['annotations'])

In [8]:
yt_p = json.load(open('429_train.json', 'r'))
yt_p = pd.DataFrame(yt_p)

yv_p = json.load(open('429_val.json', 'r'))
yv_p = pd.DataFrame(yv_p)

In [5]:
yt_g.shape, yv_g.shape

((100575, 6), (33145, 6))

In [9]:
yt_p.shape, yv_p.shape

((22866, 2), (7542, 2))

In [28]:
yt_g[yt_g.caption.str.lower().str.contains(QUALITY_ISSUE_STR)].shape, yt_p[yt_p.caption.str.lower().str.contains(QUALITY_ISSUE_STR)].shape

((85, 6), (217, 2))

In [27]:
yv_g[yv_g.caption.str.lower().str.contains(QUALITY_ISSUE_STR)].shape, yv_p[yv_p.caption.str.lower().str.contains(QUALITY_ISSUE_STR)].shape

((33, 6), (108, 2))

In [ ]:
yp = '429_val.json'
yp = json.load(open(yp, 'r'))
len(yp)
yp = pd.DataFrame(yp)
yp

In [ ]:
yt = '../data/val.json'
yt = json.load(open(yt, 'r'))

In [ ]:
yt = pd.DataFrame(yt['annotations'])
yt = yt[yt.image_id.isin(yp.image_id)]

In [ ]:
PIC_QUALITY_ISSUE = 'Quality issues are too severe to recognize visual content.'
yt = yt[yt.caption != PIC_QUALITY_ISSUE]
yt = yt[~yt.is_rejected]

In [ ]:
yp.shape, yt.shape

In [ ]:
from collections import defaultdict
gt = defaultdict(list)
for i, row in yt.iterrows():
    gt[row['image_id']].append(row['caption'])

In [ ]:
predictions = yp.to_dict('record')

In [ ]:
s, scores = scorer.compute_score(gt, predictions)

In [ ]:
s

In [ ]:
len(scores)

In [ ]:
yp['cider'] = scores

In [ ]:
yp.cider.describe()

In [ ]:
yp.to_csv('val_cider.csv', index=False)

In [ ]:
m = yp.merge(yt, on='image_id', suffixes=('_pred', '_true'))

In [ ]:
m.to_csv('val_m.csv', index=False)

In [ ]:
m.head()

In [ ]:
m[m.text_detected].cider.mean(), m[~m.text_detected].cider.mean()

In [ ]:
m['p_len'] = m.caption_pred.str.len()

In [ ]:
m['t_len'] = m.caption_true.str.len()

In [ ]:
m

In [ ]:
m['len_diff'] = m.t_len - m.p_len

In [ ]:
m[m['len_diff'] > 27]

In [ ]:
low = m[m.cider<0.4]

In [ ]:
m.cider.describe()

In [ ]:
high = m[m.cider > 0.83]

In [ ]:
high.describe()

In [ ]:
low.describe()

In [ ]:
m[m.caption_pred.str.contains('quality issue|blurry')].sort_values('cider')

In [ ]:
m[m.caption_pred.str.contains('quality issue|blurry')]['cider'].describe()